In [16]:
import sys
import regex
import cgatcore.iotools as iotools
import pysam
import logging
import collections
import os.path

In [24]:
tab = str.maketrans("ACTG", "TGAC")

def reverse_complement_table(seq):
    return seq.translate(tab)[::-1]

In [29]:
outfile = iotools.open_file('revcomp.fastq', "w")
log =  iotools.open_file("revcomp" + ".log","w")
n = 0
y = 0
with pysam.FastxFile('pooled_cDNA.fastq.gz') as fh:
    
    for record in fh:
        y +=1
        if len(record.sequence) < 300:
            pass
        else:
            seq = record.sequence[30:500]
            m=regex.findall("(TTTTTTTTTTTTTTTTTTTT){e<=0}", str(seq))
            if m:
                n +=1
                sequence = reverse_complement_table(str(record.sequence))
                outfile.write("@%s\n%s\n+\n%s\n" % (record.name, sequence, record.quality))
            else:
                seq = record.sequence[-30:-500:]
                m=regex.findall("(AAAAAAAAAAAAAAAAAAAA){e<=0}", str(seq))
                if m:
                    n +=1
                    outfile.write("@%s\n%s\n+\n%s\n" % (record.name, record.sequence, record.quality))
                else:
                    pass
                    
log.write("The number of total reads with polyA: %s\n" %(n))
log.write("The number of total reads is: %s\n" %(y))
log.write("The number of total recovered percent is: %s\n" %((n/y)*100))

log.close()
outfile.close()

In [26]:
def allCharactersSame(s) :
    n = len(s)
    for i in range(1, n) :
        if s[i] != s[0] :
            return False
 
    return True

def remove_point_mutations(umi):
    
    point_remove = []
    
    trimers = [umi[i:i+3] for i in range(0, len(umi), 3)]

    n = 0
    error_counter = 0
    for trimer in trimers:

        if allCharactersSame(trimer):
            point_remove.append(trimer)
        if not allCharactersSame(trimer):
            base = collections.Counter(trimer).most_common(1)[0][0]
            base = base+base+base
            point_remove.append(base)
            n += 1
    return("".join(point_remove), n)


def remove_indels(x, umi, first):
    
    substr = umi[x:x+3]
    
    if ("CCC" in substr or "GGG" in substr or "AAA" in substr or "TTT" in substr) and first:
        pass
    
#    else:
#        # correct for point mutations at beginning
#        base = collections.Counter(substr).most_common(1)[0][0]
#        substr = base+base+base
#        first = False
        
    
    if not ("CCC" in substr or "GGG" in substr or "AAA" in substr or "TTT" in substr):
        # if there is an error then check to see if the next base is a perfect match
        substr = umi[x+1:x+4]
        
        if ("CCC" in substr or "GGG" in substr or "AAA" in substr or "TTT" in substr):
            # then check to see if perfect trimer
            pass

        elif not ("CCC" in substr or "GGG" in substr or "AAA" in substr or "TTT" in substr):
            # if not then check to see if there is a 
            substr = umi[x+2:x+5]

            
            
    return(substr)


In [37]:
dict = {'AAATTTGGGCCC': '1',
        'TTTCCCAAAGGG': '2',
        'GGGAAACCCTTT': '3',
        'CCCGGGTTTAAA': '4',
        'AAACCCGGGAAA': '5',
        'TTTGGGAAATTT': '6',
        'GGGTTTCCCGGG': '7',
        'CCCAAATTTCCC': '8',
        'AAAGGGAAAGGG': '9',
        'TTTAAATTTAAA': '10',
        'GGGCCCGGGCCC': '11',
        'CCCTTTCCCTTT': '12'}




with pysam.FastxFile('revcomp.fastq') as fh:
    
    for record in fh:
        
        seq_nano = record.sequence

        m=regex.finditer("(AAGCAGTGGT){e<=0}", str(seq_nano))
        
        
        for i in m:
            barcode = seq_nano[int(i.start())-12:int(i.start())]
            barcode = remove_point_mutations(barcode)[0]
            barcode = barcode[::3]
            if barcode == 'ATGC':
                fname = "Sample1.fastq"
            elif barcode == 'TCAG':
                fname = "Sample2.fastq"
            elif barcode == 'GACT':
                fname = "Sample3.fastq"
            elif barcode == 'CGTA':
                fname = "Sample4.fastq"
            elif barcode == 'ACGA':
                fname = "Sample5.fastq"
            elif barcode == 'TGAT':
                fname = "Sample6.fastq"
            elif barcode == 'GTCG':
                fname = "Sample7.fastq"
            elif barcode == 'CATC':
                fname = "Sample8.fastq"
            else:
                break
            
            if os.path.exists(fname):
                read = iotools.open_file(fname,"a")
            else:
                read = iotools.open_file(fname,"w")
            
            read.write("@%s\n%s\n+\n%s\n" % (record.name, record.sequence, record.quality))
            
            read.close()